In [1]:
!pip install lxml
!pip install geocoder
!pip install folium
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import geocoder 
import folium # map rendering library
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

     |████████████████████████████████| 102kB 8.5MB/s ta 0:00:011
     |████████████████████████████████| 92kB 5.8MB/s eta 0:00:011


Read the table on the website and assign it to dataframe df

In [2]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = tables[0]
df.head(20)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Downtown Toronto,Queen's Park


Drop all rows with value == "Not assigned" of column Borough

In [3]:
df = df[df["Borough"]!= "Not assigned"]
df.head(20)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


Replace all Neighbohoods with value "Not assigned" with it Borough


In [4]:
df.loc[df["Neighborhood"]=="Not assigned","Neighborhood"]= df.loc[df["Neighborhood"]=="Not assigned","Borough"]
df.head(20)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


Save all Postcode which are multiple in the table to a dictionary. The postcode is the key and the Borough is the value

In [5]:
postCodes = df["Postcode"].values.T.tolist()

postCodesNTimes = {}
for i in range(1,len(postCodes)):
    postCode = postCodes[i]
    for j in  range(i+1,len(postCodes)):
            if postCode == postCodes[j]:
                postCodesNTimes[postCode]=df.iloc[j,1]
    
postCodesNTimes

{'M6A': 'North York',
 'M1B': 'Scarborough',
 'M4B': 'East York',
 'M5B': 'Downtown Toronto',
 'M9B': 'Etobicoke',
 'M1C': 'Scarborough',
 'M3C': 'North York',
 'M9C': 'Etobicoke',
 'M1E': 'Scarborough',
 'M3H': 'North York',
 'M5H': 'Downtown Toronto',
 'M6H': 'West Toronto',
 'M2J': 'North York',
 'M3J': 'North York',
 'M5J': 'Downtown Toronto',
 'M6J': 'West Toronto',
 'M1K': 'Scarborough',
 'M3K': 'North York',
 'M4K': 'East Toronto',
 'M5K': 'Downtown Toronto',
 'M6K': 'West Toronto',
 'M1L': 'Scarborough',
 'M2L': 'North York',
 'M4L': 'East Toronto',
 'M5L': 'Downtown Toronto',
 'M6L': 'North York',
 'M1M': 'Scarborough',
 'M2M': 'North York',
 'M5M': 'North York',
 'M6M': 'York',
 'M9M': 'North York',
 'M1N': 'Scarborough',
 'M6N': 'York',
 'M1P': 'Scarborough',
 'M5P': 'Central Toronto',
 'M6P': 'West Toronto',
 'M1R': 'Scarborough',
 'M5R': 'Central Toronto',
 'M6R': 'West Toronto',
 'M9R': 'Etobicoke',
 'M5S': 'Downtown Toronto',
 'M6S': 'West Toronto',
 'M1T': 'Scarborough'

Go over the dictionary with the multiple postcodes

In [6]:
dfPostCodesUnique = df

for postCodeNTimes in postCodesNTimes:    
    dfPostCode = df[df["Postcode"]==postCodeNTimes]
    neighborhoods = dfPostCode["Neighborhood"].values.T.tolist() # save all neighborhoods of a postcode postCodeNTimes to list neighborhoods
    neighborhoods = ",".join(neighborhoods) #create a string from the list neighborhoods with speperator comma.
    dfPostCodesUnique =dfPostCodesUnique[dfPostCodesUnique.Postcode != postCodeNTimes] # delete all rows with the postcode postCodeNTimes
    #create a new dataframe with the joined neighborhoods
    dfJoin = pd.DataFrame([[postCodeNTimes,postCodesNTimes[postCodeNTimes],neighborhoods]], columns=["Postcode", "Borough", "Neighborhood"])
    #append the new dataframe to dfPostCodesUnique
    dfPostCodesUnique=dfPostCodesUnique.append(dfJoin, ignore_index = True)

dfPostCodesUnique= dfPostCodesUnique.rename(columns={"Postcode":"PostalCode"})
dfPostCodesUnique.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M7A,Queen's Park,Queen's Park
4,M9A,Downtown Toronto,Queen's Park
5,M3B,North York,Don Mills North
6,M6B,North York,Glencairn
7,M4C,East York,Woodbine Heights
8,M5C,Downtown Toronto,St. James Town
9,M6C,York,Humewood-Cedarvale


In [7]:
dfPostCodesUnique.tail(20)

,PostalCode,Borough,Neighborhood
83,M1R,Scarborough,"Maryvale,Wexford"
84,M5R,Central Toronto,"The Annex,North Midtown,Yorkville"
85,M6R,West Toronto,"Parkdale,Roncesvalles"
86,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
87,M5S,Downtown Toronto,"Harbord,University of Toronto"
88,M6S,West Toronto,"Runnymede,Swansea"
89,M1T,Scarborough,"Clarks Corners,Sullivan,Tam O'Shanter"
90,M4T,Central Toronto,"Moore Park,Summerhill East"
91,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market"
92,M1V,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel..."


In [8]:
dfPostCodesUnique.shape

(103, 3)

In [22]:
# The code was removed by Watson Studio for sharing.

In [23]:
dfPostCodesUnique.sort_values("PostalCode",ascending=True, inplace=True)
dfPostCodesUnique = dfPostCodesUnique.reset_index(drop=True)


geoCode.head(20)

,Unnamed: 0,Postal Code,Latitude,Longitude
0,0,M1B,43.806686,-79.194353
1,1,M1C,43.784535,-79.160497
2,2,M1E,43.763573,-79.188711
3,3,M1G,43.770992,-79.216917
4,4,M1H,43.773136,-79.239476
5,5,M1J,43.744734,-79.239476
6,6,M1K,43.727929,-79.262029
7,7,M1L,43.711112,-79.284577
8,8,M1M,43.716316,-79.239476
9,9,M1N,43.692657,-79.264848


In [24]:
dfPostCodesUnique["Latitude"]= geoCode["Latitude"]
dfPostCodesUnique["Longitude"]= geoCode["Longitude"]
dfPostCodesUnique

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [25]:

latitude = dfPostCodesUnique["Latitude"].mean()
longitude = dfPostCodesUnique["Longitude"].mean()
map_toronto= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfPostCodesUnique['Latitude'], dfPostCodesUnique['Longitude'], dfPostCodesUnique['Borough'], dfPostCodesUnique['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [26]:
CLIENT_ID = "WKQBB1DZI22C3IPI4ADEQIKOA1PYAITSAN3NIY3VLMFTNSMG" # your Foursquare ID
CLIENT_SECRET = 'LTVCWEBBOCNZYZX4K35HI4Z25HE2XQTT4GJ2DWOK21HIPIOC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WKQBB1DZI22C3IPI4ADEQIKOA1PYAITSAN3NIY3VLMFTNSMG
CLIENT_SECRET:LTVCWEBBOCNZYZX4K35HI4Z25HE2XQTT4GJ2DWOK21HIPIOC


In [27]:
neighborhoodsOnlyToronto = []
for i in range(0,dfPostCodesUnique.shape[0]):
    if "Toronto" in dfPostCodesUnique.iloc[i,1]:
        neighborhoodsOnlyToronto.append(True)
    else:
        neighborhoodsOnlyToronto.append(False)
dfneighborhoodsOnlyToronto=dfPostCodesUnique[neighborhoodsOnlyToronto]  
dfneighborhoodsOnlyToronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [28]:
latitude = dfneighborhoodsOnlyToronto["Latitude"].mean()
longitude = dfneighborhoodsOnlyToronto["Longitude"].mean()
map_toronto2= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfneighborhoodsOnlyToronto['Latitude'], dfneighborhoodsOnlyToronto['Longitude'], dfneighborhoodsOnlyToronto['Borough'], dfneighborhoodsOnlyToronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

In [29]:
firstcan=dfneighborhoodsOnlyToronto.loc[70, 'Neighborhood']
firstcan

'First Canadian Place,Underground city'

In [ ]:
neighborhood_latitude = dfneighborhoodsOnlyToronto.loc[70, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfneighborhoodsOnlyToronto.loc[70, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfneighborhoodsOnlyToronto.loc[70, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

In [ ]:
results = requests.get(url).json()

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()